# Práctica 2: Búsqueda adversaria y optimización

Equipo de trabajo:

    Macal Cruz Brandon Brayan, 318085470

    Nieto Calderón Camilo, 317521861

    Ortega Medina David, 319111866
	
    Alvarado Camacho Andrea, 318064343

# 1. Programa el algoritmo de α −β aplicado al problema del juego del gato. Realiza lo siguiente:
 ### a)Construye el juego del gato con jugador MAX con tiradas ‘x’ y jugador MIN con tiradas ‘o’. La funcion de utilidad dada como: +1 si max gana, -1 si max pierde , 0 si hay empate

Este código define la clase Gato que modela el juego del gato (o tres en raya) entre dos jugadores: el jugador MAX que utiliza las tiradas ‘x’ (representado como 1) y el jugador MIN que utiliza las tiradas ‘o’ (representado como -1).

La clase inicializa un tablero de 3x3, donde cada celda se puede encontrar vacía (0), ocupada por el jugador MAX (1), o ocupada por el jugador MIN (-1). El constructor permite establecer un estado inicial del tablero, que por defecto es completamente vacío.

**`__str__`**: Devuelve una representación en cadena del tablero, mostrando ‘x’ para el jugador MAX representado como en 1 en un arreglo, ‘o’ para el jugador MIN representado como -1, y un espacio en blanco para las posiciones vacías representado en el arreglo como 0. 

**`change_player`**: Cambia el turno del jugador. Cuando este método es llamado, el jugador actual se multiplica por -1, alternando entre 1 y -1.

**`actions`**: Este método devuelve una lista de todas las posiciones vacías en el tablero, que representan las acciones posibles que un jugador puede tomar en su turno.

**`result`**: Dada una acción y el jugador actual, este método devuelve un nuevo estado del tablero después de aplicar la acción. Utiliza el método `copy` de NumPy para asegurarse de que el estado se modifica de manera adecuada sin afectar el estado original.

**`is_terminal`**: Verifica si el juego ha terminado, devolviendo `True` si el juego ha llegado a su fin (cuando hay un ganador o un empate).

**`utility`**: Este método calcula la utilidad del estado actual del juego:

- Devuelve +1 si el jugador MAX (1) gana.
- Devuelve -1 si el jugador MIN (-1) gana.
- Devuelve 0 si hay un empate (es decir, si el tablero está lleno y no hay ganador).
- Devuelve `None` si el juego aún continúa.


In [ ]:
import numpy as np

class Gato:
    def __init__(self, board=None):
        self.board = np.zeros((3, 3), dtype=int) if board is None else board
        self.player = 1  # MAX (1) empieza siempre

    def __str__(self):
        return '\n'.join(['|'.join(['x' if cell == 1 else 'o' if cell == -1 else ' ' for cell in row]) for row in self.board])

    def change_player(self):
        self.player *= -1

    def actions(self, state):
        """Devuelve una lista de todas las posiciones vacías (acciones posibles)."""
        return [(i, j) for i in range(3) for j in range(3) if state[i, j] == 0]

    def result(self, state, action, player):
        """Devuelve un nuevo estado después de aplicar la acción del jugador especificado."""
        new_state = state.copy()
        new_state[action[0], action[1]] = player  
        return new_state

    def is_terminal(self, state):
        """Verifica si el juego ha terminado y devuelve True si es así."""
        return self.utility(state) is not None

    def utility(self, state):
        """Devuelve 1 si gana MAX, -1 si gana MIN, 0 si es empate, y None si el juego sigue."""
        for row in state:
            if abs(sum(row)) == 3:
                return 1 if row[0] == 1 else -1
        for col in state.T:
            if abs(sum(col)) == 3:
                return 1 if col[0] == 1 else -1
        if abs(state[0, 0] + state[1, 1] + state[2, 2]) == 3:
            return 1 if state[0, 0] == 1 else -1
        if abs(state[0, 2] + state[1, 1] + state[2, 0]) == 3:
            return 1 if state[0, 2] == 1 else -1
        if all(state[row][col] != 0 for row in range(3) for col in range(3)):
            return 0
        return None

### b) Genera la funcion para  ́ α −beta y estima los valores y acciones para los siguientes estados (en todos los casos TO-MOVE = MAX):

### Funciones del Algoritmo Alpha-Beta

#### `max_value(game, state, alpha, beta)`
- **Propósito**: Calcula el valor máximo que el jugador MAX (jugador 1) puede obtener desde un estado dado.
- **Funcionamiento**:
  - Verifica si el estado es terminal y retorna su utilidad.
  - Inicializa `v` con `-∞` y busca todas las acciones posibles.
  - Para cada acción, aplica la acción y llama a `min_value`.
  - Actualiza `v` y `move` si encuentra un valor mejor.
  - Realiza poda beta si `alpha` es mayor o igual a `beta`.

#### `min_value(game, state, alpha, beta)`
- **Propósito**: Calcula el valor mínimo que el jugador MIN (jugador -1) puede obtener desde un estado dado.
- **Funcionamiento**:
  - Verifica si el estado es terminal y retorna su utilidad.
  - Inicializa `v` con `∞` y busca todas las acciones posibles.
  - Para cada acción, aplica la acción y llama a `max_value`.
  - Actualiza `v` y `move` si encuentra un valor mejor.
  - Realiza poda alpha si `alpha` es mayor o igual a `beta`.

#### `alphabeta(game)`
- **Propósito**: Inicia el algoritmo de búsqueda alpha-beta dependiendo de quién es el jugador actual.
- **Funcionamiento**: Llama a `max_value` si es el turno de MAX, o a `min_value` si es el turno de MIN.

#### `play_game(board)`
- **Propósito**: Juega una partida del gato desde un estado inicial dado.
- **Funcionamiento**:
  - Inicializa el juego con el estado del tablero.
  - Muestra el tablero y alterna entre los jugadores.
  - Aplica los movimientos calculados por `alphabeta`.
  - Muestra el tablero después de cada movimiento.
  - Termina el juego cuando se alcanza un estado terminal.

### Ejecución
Se ejecuta el juego para varios estados iniciales definidos en `initial_boards`.



In [ ]:

def max_value(game, state, alpha, beta):
    utility = game.utility(state)
    if utility is not None:
        return utility, None
    v, move = -np.inf, None
    for action in game.actions(state):
        new_state = game.result(state, action, 1)  # Aplicar acción como MAX (jugador 1)
        v2, _ = min_value(game, new_state, alpha, beta)
        if v2 > v:
            v, move = v2, action
        alpha = max(alpha, v)
        if alpha >= beta:
            break
    return v, move

def min_value(game, state, alpha, beta):
    utility = game.utility(state)
    if utility is not None:
        return utility, None
    v, move = np.inf, None
    for action in game.actions(state):
        new_state = game.result(state, action, -1)  # Aplicar acción como MIN (jugador -1)
        v2, _ = max_value(game, new_state, alpha, beta)
        if v2 < v:
            v, move = v2, action
        beta = min(beta, v)
        if alpha >= beta:
            break
    return v, move

def alphabeta(game):
    """Inicia el algoritmo de alpha-beta en función de quién es el jugador actual."""
    state = game.board
    if game.player == 1:
        return max_value(game, state, -np.inf, np.inf)
    else:
        return min_value(game, state, -np.inf, np.inf)

def play_game(board):
    """Jugar el juego del gato con alpha-beta desde un estado inicial dado."""
    game = Gato(board)
    print(f"Jugador {game.player} está jugando")
    print(game)
    print()  # Línea en blanco para separar las jugadas

    while not game.is_terminal(game.board):
        value, move = alphabeta(game)
        if move is not None:
            game.board = game.result(game.board, move, game.player)
            game.change_player()
            
            # Mostrar el tablero después de cada movimiento con el jugador actual
            print(f"Jugador {game.player} está jugando")
            print(game)
            print()  # Línea en blanco para separar las jugadas
        else:
            break

    print("Juego terminado")
    print(game)

print("Caso 1")
play_game(initial_boards[0])

print("\n")
print("\n")

print("Caso 2")
play_game(initial_boards[1])

print("\n")
print("\n")

print("Caso 3")
play_game(initial_boards[2])

print("\n")
print("\n")

print("Caso 4")
play_game(initial_boards[3])

# 2. Programa un algoritmo genetico para el problema de las   n reinas. Sigue los siguientes pasos: 
## a) Construye el problema de las n reinas donde se colocan n piezas reinas en un tablero de ajedrez de tamano 8

**`__init__`**: Este método inicializa un tablero de ajedrez de tamaño `size` (por defecto, 8) y coloca las reinas en posiciones aleatorias utilizando una permutación de los índices de las columnas.

**`__str__`**: Este método devuelve una representación en cadena del tablero, mostrando `1` en las posiciones ocupadas por las reinas y `0` en las posiciones vacías. 

**`draw`**: Este método utiliza Matplotlib para dibujar el tablero, mostrando las reinas en negro sobre un fondo blanco.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(12345)

class Board:

    def __init__(self, size=8):
        self.size = size
        self.queens = np.random.permutation(size)

    def __str__(self):
        """Muestra el tablero en formato de texto."""
        board = np.zeros((self.size, self.size))
        for row, col in enumerate(self.queens):
            board[row][col] = 1
        return str(board)

    def draw(self):
        """Dibuja el tablero con las reinas."""
        board = np.zeros((self.size, self.size))
        for row, col in enumerate(self.queens):
            board[row][col] = 1
        plt.imshow(board, cmap='Greys', extent=[0, self.size, 0, self.size])
        plt.xticks([])
        plt.yticks([])
        plt.show()

b) Define la funcion fitness como el negativo del total de reinas que se estan amenazando entre si. El valor maximo debe ser 0.


In [ ]:
def calculate_fitness(queens):
    n = len(queens)
    conflicts = 0
    
    for i in range(n):
        for j in range(i + 1, n):
            if queens[i] == queens[j] or abs(queens[i] - queens[j]) == abs(i - j):
                conflicts += 1

    return -conflicts

c) Define la funcion de seleccion con los siguientes metodos: 
* 1) Ruleta

In [ ]:
def roulette_wheel_selection(population, fitnesses, num_selected=2):
    total_fitness = np.sum(fitnesses)
    probabilities = fitnesses / total_fitness
    selected_indices = np.random.choice(len(population), size=num_selected, replace=False, p=probabilities)
    selected = [population[i] for i in selected_indices]
    return selected

* 2) Aleatorio

In [ ]:
def random_selection(population, num_selected=2):
    selected_indices = np.random.choice(len(population), size=num_selected, replace=False)
    selected = [population[i] for i in selected_indices]
    return selected

* 3)  Ranking 

In [ ]:
def ranking_selection(population, fitnesses, num_selected=2):
    sorted_indices = np.argsort(fitnesses)[::-1]
    sorted_population = [population[i] for i in sorted_indices]
  
    ranks = np.arange(1, len(population) + 1)
    probabilities = ranks / np.sum(ranks)
    
    selected_indices = np.random.choice(len(sorted_population), size=num_selected, replace=False, p=probabilities)
    selected = [sorted_population[i] for i in selected_indices]
    return selected

d) Define la funcion de reproduccion con los siguientes metodos: 
* 1) 1 punto

In [ ]:
def one_point_crossover(parent1, parent2):
    size = len(parent1)
    crossover_point = np.random.randint(1, size)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

* 2) n puntos

In [ ]:
def n_point_crossover(parent1, parent2, n_points=2):
    size = len(parent1)
    crossover_points = sorted(np.random.choice(range(1, size), n_points, replace=False))
    child1, child2 = parent1.copy(), parent2.copy()
    swap = False
    current_index = 0
    
    for point in crossover_points:
        if swap:
            child1[current_index:point], child2[current_index:point] = child2[current_index:point], child1[current_index:point]
        swap = not swap
        current_index = point

    if swap:
        child1[current_index:], child2[current_index:] = child2[current_index:], child1[current_index:]
    
    return child1, child2

* 3) uniforme

In [ ]:
def uniform_crossover(parent1, parent2):
    size = len(parent1)
    child1, child2 = parent1.copy(), parent2.copy()
    
    for i in range(size):
        if np.random.rand() > 0.5:
            child1[i], child2[i] = child2[i], child1[i]
    
    return child1, child2

e) Define la funcion de mutacion con los siguientes metodos: 
* 1) Flipping

In [ ]:
def flipping_mutation(queens):
    mutated_queens = np.array(queens.copy())
    size = len(mutated_queens)

    row = np.random.randint(0, size)
    
    current_col = mutated_queens[row]
    new_col = np.random.choice([col for col in range(size) if col != current_col])

    mutated_queens[row] = new_col
    
    return mutated_queens

* 2) Intercambio

In [ ]:
def swap_mutation(queens):
    """Mutación por intercambio: Intercambia las posiciones de dos reinas."""
    mutated_queens = np.array(queens.copy())
    size = len(mutated_queens)

    row1, row2 = np.random.choice(size, size=2, replace=False)
    
    mutated_queens[row1], mutated_queens[row2] = mutated_queens[row2], mutated_queens[row1]
    
    return mutated_queens


* 3) Reverso

In [ ]:
def reverse_mutation(queens):
    mutated_queens = np.array(queens.copy())
    size = len(mutated_queens)
 
    point1, point2 = sorted(np.random.choice(size, size=2, replace=False))
   
    mutated_queens[point1:point2 + 1] = mutated_queens[point1:point2 + 1][::-1]
    
    return mutated_queens

f) Usa un reemplazo debil en cada caso.

In [ ]:
def weak_replacement(population, fitnesses, offspring, offspring_fitnesses):
    """
    Realiza un reemplazo débil. Reemplaza a los peores individuos de la población con los hijos generados.
    """
    combined_population = np.concatenate((population, offspring), axis=0)
    combined_fitnesses = np.concatenate((fitnesses, offspring_fitnesses), axis=0)
    
    # Ordenamos a la población combinada según fitness, de mayor a menor (mejores primero)
    sorted_indices = np.argsort(combined_fitnesses)[::-1]
    
    # Mantenemos a los mejores individuos de la población combinada
    new_population = [combined_population[i] for i in sorted_indices[:len(population)]]
    new_fitnesses = [combined_fitnesses[i] for i in sorted_indices[:len(population)]]
    
    return np.array(new_population), np.array(new_fitnesses)

def genetic_algorithm(population_size=100, board_size=8, max_generations=1000, crossover_method='one_point', 
                      mutation_method='flipping', selection_method='roulette', mutation_rate=0.1):
    """
    Ejecuta el algoritmo genético para resolver el problema de las n reinas.
    """
    # Inicializamos la población aleatoria
    population = [Board(board_size).queens for _ in range(population_size)]
    fitnesses = np.array([calculate_fitness(individual) for individual in population])

    # Iteramos a través de las generaciones
    for generation in range(max_generations):
        print(f"Generación {generation} - Mejor fitness: {np.max(fitnesses)}")
        
        if np.max(fitnesses) == 0:
            print("Solución encontrada!")
            break
        
        # Selección de padres
        if selection_method == 'roulette':
            parents = roulette_wheel_selection(population, fitnesses)
        elif selection_method == 'random':
            parents = random_selection(population)
        elif selection_method == 'ranking':
            parents = ranking_selection(population, fitnesses)
        
        # Cruce para generar hijos
        if crossover_method == 'one_point':
            offspring = one_point_crossover(parents[0], parents[1])
        elif crossover_method == 'n_point':
            offspring = n_point_crossover(parents[0], parents[1], n_points=2)
        elif crossover_method == 'uniform':
            offspring = uniform_crossover(parents[0], parents[1])

        offspring = list(offspring)
        
        # Aplicamos mutación a los hijos
        for i in range(len(offspring)):
            if np.random.rand() < mutation_rate:
                if mutation_method == 'flipping':
                    offspring[i] = flipping_mutation(offspring[i])
                elif mutation_method == 'swap':
                    offspring[i] = swap_mutation(offspring[i])
                elif mutation_method == 'reverse':
                    offspring[i] = reverse_mutation(offspring[i])
        
        # Calculamos el fitness de los hijos
        offspring_fitnesses = np.array([calculate_fitness(ind) for ind in offspring])
        
        # Reemplazo débil
        population, fitnesses = weak_replacement(population, fitnesses, offspring, offspring_fitnesses)

    # Retornamos la mejor solución encontrada
    best_solution_index = np.argmax(fitnesses)
    return population[best_solution_index], fitnesses[best_solution_index]

if __name__ == "__main__":
    best_solution, best_fitness = genetic_algorithm(
        population_size=8, 
        board_size=10, 
        max_generations=1000, 
        crossover_method='one_point', 
        mutation_method='flipping', 
        selection_method='roulette', 
        mutation_rate=0.1
    )
    
    print(f"Mejor solución: {best_solution}")
    print(f"Mejor fitness: {best_fitness}")

g) Evalua el algoritmo genetico sobre el problema de las 10 reinas, tomando en cuenta: 
* 1) El tamano de la poblacion debe ser de 8. 
* 2) Se correra el algoritmo un m  ́ aximo de 10 iteraciones. 
* 3) Regresa el valor fitness del candidato mas apto para cada una de las posibles combinaciones de los operadores geneticos. 

In [ ]:
def evaluate_10_queens():
    # Definir las combinaciones de operadores genéticos
    selection_methods = ['roulette', 'random', 'ranking']
    crossover_methods = ['one_point', 'n_point', 'uniform']
    mutation_methods = ['flipping', 'swap', 'reverse']
    
    # Parámetros del problema
    board_size = 10  # 10 reinas
    population_size = 8  # Tamaño de la población
    max_generations = 10  # Máximo de 10 iteraciones
    
    # Evaluar todas las combinaciones de operadores genéticos
    for selection in selection_methods:
        for crossover in crossover_methods:
            for mutation in mutation_methods:
                print(f"Evaluando combinación: Selección: {selection}, Cruce: {crossover}, Mutación: {mutation}")
                
                # Ejecutar el algoritmo genético con la combinación actual
                best_solution, best_fitness = genetic_algorithm(
                    population_size=population_size,
                    board_size=board_size,
                    max_generations=max_generations,
                    crossover_method=crossover,
                    mutation_method=mutation,
                    selection_method=selection,
                    mutation_rate=0.1  # Tasa de mutación fija
                )
                
                # Si se encuentra la solución perfecta (fitness == 0)
                if best_fitness == 0:
                    print(f"Solución perfecta encontrada con combinación: Selección: {selection}, Cruce: {crossover}, Mutación: {mutation}")
                    print(f"Mejor solución (posiciones de las reinas): {best_solution}")
                    
                    # Imprimir el tablero con la mejor solución
                    board = Board(board_size)
                    board.queens = best_solution  # Colocar la mejor solución en el tablero
                    print("Tablero de ajedrez:")
                    print(board)  # Mostrar el tablero en texto
                    
                    # Dibujar el tablero
                    board.draw()
                    
                    # Salir del bucle ya que se encontró la solución perfecta
                    return

if __name__ == "__main__":
    evaluate_10_queens()